#### Vicsek Model Simulations

**Authors:** Subhanik Purkayastha and Dhananjay Bhaskar

**Last Modified:** Dec 08, 2018

In [28]:
import os
import math
import numpy as np
import random as nr
from IPython.display import HTML
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot

from joblib import Parallel, delayed
import multiprocessing

init_notebook_mode(connected=True)

%matplotlib inline

In [33]:
def findAngle(events, cid):
    return pos[events][cid]['a']

def getColor(events, cid):
    return pos[events][cid]['c']

def averageTheta(event, cell):
    theta = pos[event][cell]['a']
    count = 1.0
    for cid in range(N):
        if(pos[event][cell]['x'] + attraction > 1000):
            if((pos[event][cid]['x'] < (-1000+(attraction-(1000-pos[event][cell]['x'])))) and (abs(pos[event][cid]['y'] - pos[event][cell]['y']) <= attraction)):
                thetha=theta+pos[event][cid]['a']
                count+=count+1.0
        if(pos[event][cell]['x'] - attraction < -1000):
            if((pos[event][cid]['x'] > (1000-(attraction+(-1000-pos[event][cell]['x'])))) and (abs(pos[event][cid]['y'] - pos[event][cell]['y']) <= attraction)):
                theta=theta+pos[event][cid]['a']
                count=count+1.0
        if(pos[event][cell]['y'] + attraction > 1000):
            if((pos[event][cid]['y'] < (-1000+(attraction-(1000-pos[event][cell]['y'])))) and (abs(pos[event][cid]['x'] - pos[event][cell]['x']) <= attraction)):
                thetha=theta+pos[event][cid]['a']
                count=count+1.0
        if(pos[event][cell]['y'] - attraction < -1000):
            if((pos[event][cid]['y']>(1000-(attraction+(-1000-pos[event][cell]['y'])))) and (abs(pos[event][cid]['x'] - pos[event][cell]['x']) <= attraction)):
                theta=theta+pos[event][cid]['a']
                count=count+1.0
        if((abs(pos[event][cid]['x'] - pos[event][cell]['x']) <= attraction) and (abs(pos[event][cid]['y'] - pos[event][cell]['y']) <= attraction)):
            theta=theta + pos[event][cid]['a']
            count=count+1.0
    return (theta/count)
            
def getXPosition(cid, t):
    x_data = list()
    if cid in pos[t].keys():
        x_data.append(pos[t][cid]['x'])
    return x_data

def getYPosition(cid, t):
    y_data = list()
    if cid in pos[t].keys():
        y_data.append(pos[t][cid]['y'])
    return y_data

def getName(num):
    numZeros = 5 - len(repr(num))
    name = ""
    for i in range(numZeros):
        name += '0'
    name += repr(num)
    name += ".png"
    return name

def areInteracting(cell, event):
    for cid in range(N):
        if ((abs(pos[event][cid]['x'] - pos[event][cell]['x']) <= attraction) and (abs(pos[event][cid]['y'] - pos[event][cell]['y']) <= attraction)):
            plt.plot([pos[event][cid]['x'], pos[event][cell]['x']], [pos[event][cid]['y'], pos[event][cell]['y']], 'k-')
            
def showTrail(cell, event):
    if (event <= 10):
        for i in range(event):
            plt.plot([pos[i][cell]['x'], pos[i+1][cell]['x']], [pos[i][cell]['y'], pos[i+1][cell]['y']], 'k--')
    else:
        for i in range(10):
            plt.plot([pos[event-i-1][cell]['x'], pos[event-i][cell]['x']], [pos[event-i-1][cell]['y'], pos[event-i][cell]['y']], 'g--')

def setInitialCircle():
    x = list()
    y = list()
    for n in range(N):
        angle = nr.uniform(0,1)*(math.pi*2)
        x.append(800*math.cos(angle));
        y.append(800*math.sin(angle));
    
    for i in range(N):
        firstPos[i] = dict()
        for j in ['x', 'y', 'a', 'c']:
            if (j == 'c'):
                firstPos[i][j] = 'b'
            if (j == 'x'):
                firstPos[i][j] = x[i];
            if (j == 'y'):
                firstPos[i][j] = y[i];
            if (j == 'a'):
                if(firstPos[i]['x'] > 0):
                    firstPos[i][j] = math.pi+math.atan((firstPos[i]['y'])/(firstPos[i]['x']))
                else:
                    firstPos[i][j] = math.atan((firstPos[i]['y'])/(firstPos[i]['x']))
                        
def setOrbitingCircle():
    angle = 0
    delta_theta = (2*math.pi)/N
    pos = list()
    for n in range(N):
        pos.append([800*math.cos(angle), 800*math.sin(angle)]);
        angle = angle+delta_theta
    
    for i in range(N-1):
        firstPos[i] = dict()
        for j in ['x', 'y', 'a', 'c']:
            if (j == 'c'):
                firstPos[i][j] = 'b'
            if (j == 'x'):
                firstPos[i][j] = pos[i][0];
            if (j == 'y'):
                firstPos[i][j] = pos[i][1];
            if (j == 'a'):
                if(firstPos[i]['y'] > 0 and firstPos[i]['x'] > 0):
                    firstPos[i][j] = math.atan((pos[i+1][1]-pos[i][1])/(pos[i+1][0]-pos[i][0]))
                elif(firstPos[i]['y'] > 0 and firstPos[i]['x'] < 0):
                    firstPos[i][j] = (math.pi/2)+math.atan((pos[i+1][0]-pos[i][0])/(pos[i+1][1]-pos[i][1]))
                elif(firstPos[i]['y'] < 0 and firstPos[i]['x'] > 0):
                    firstPos[i][j] = math.atan((pos[i+1][0]-pos[i][0])/(pos[i+1][1]-pos[i][1]))-(math.pi)
                else:
                    firstPos[i][j] = math.pi+math.atan((pos[i+1][0]-pos[i][0])/(pos[i+1][1]-pos[i][1]))

def setDiagonal():
    x = np.linspace(-800, 800)
    y = np.linspace(-800, 800)
    for i in range(N):
        firstPos[i] = dict()
        for j in ['x', 'y', 'a', 'c']:
            if (j == 'c'):
                firstPos[i][j] = 'b'
            if (j == 'x'):
                firstPos[i][j] = x[i];
            if (j == 'y'):
                firstPos[i][j] = y[i];
            if (j == 'a'):
                if(firstPos[i]['x'] > 0):
                    firstPos[i][j] = math.pi+math.atan((firstPos[i]['y'])/(firstPos[i]['x']))
                else:
                    firstPos[i][j] = math.atan((firstPos[i]['y'])/(firstPos[i]['x']))

def setGrid():
    x = np.linspace(-800, 800, num=5)
    y = np.linspace(-800, 800, num=15)
    c = 0
    for row in range(y.size):
        for col in range(x.size):
            firstPos[c] = dict()
            firstPos[c]['x'] = x[col];
            firstPos[c]['y'] = y[row];
            if(firstPos[c]['x'] >= 0):
                firstPos[c]['a'] = math.pi+math.atan((firstPos[c]['y'])/(firstPos[c]['x']))
            else:
                firstPos[c]['a'] = math.atan((firstPos[c]['y'])/(firstPos[c]['x']))
            firstPos[c]['t'] = np.random.normal(100,2)
            c=c+1

def cellDivide(event, cell):
    if(pos[event][cell]['t'] <= 0):
        pos[event][N+1]['c'] = pos[event][cell]['c']
        pos[event][N+1]['x'] = pos[event][cell]['x']
        pos[event][N+1]['y'] = pos[event][cell]['y']
        pos[event][N+1]['a'] = pos[event][cell]['a']+math.pi
        pos[event][N+1]['t'] =  np.random.normal(100,2)
        pos[event][cell]['t'] = np.random.normal(100,2)
        N=N+1
    pos[event][cell]['t'] = pos[event][cell]['t']-10

In [34]:
N = 75
timePoints = 250
r = 20
attraction = 30

pos = list()

firstPos = dict()
pos.append(firstPos)

#setInitialCircle();
#setDiagonal();
setGrid();
#setOrbitingCircle();

for events in range(timePoints-1):
    nextPos = dict()
    pos.append(nextPos);
    for i in range(N):
        nextPos[i] = dict()
        for j in ['c', 'x', 'y', 'a', 't']:
            if(j=='x'):
                if(pos[events][i][j]>1000):
                    pos[events][i][j] = -1000
                    
                elif(pos[events][i][j]<-1000):
                    pos[events][i][j] = 1000
                nextPos[i][j] = pos[events][i][j]+(math.cos(pos[events][i]['a'])*r) 
            elif(j=='y'):
                if(pos[events][i][j]>1000):
                    pos[events][i][j] = -1000
                elif(pos[events][i][j]<-1000):
                    pos[events][i][j] = 1000
                nextPos[i][j] = pos[events][i][j]+(math.sin(pos[events][i]['a'])*r)
            elif (j=='c'):
                nextPos[i][j] = 'b'
            elif (j=='a'):
                nextPos[i][j] = averageTheta(events, i)
            else:
                nextPos[i][j] = cellDivide(events, i)

/home/labmember/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:144: RuntimeWarning:

divide by zero encountered in double_scalars

/home/labmember/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:144: RuntimeWarning:

invalid value encountered in double_scalars



UnboundLocalError: local variable 'N' referenced before assignment

In [18]:
num_cores = multiprocessing.cpu_count()
print num_cores

4


In [19]:
tpoints = range(timePoints) 

def createPlot(frame):
    plt.figure(figsize = (4, 3), dpi = 300)
    plt.xlim([-1000,1000])
    plt.ylim([-1000,1000])
    for cid in range(N):
        xloc = getXPosition(cid, frame)[0]
        yloc = getYPosition(cid, frame)[0]
        theta_val = findAngle(frame, cid)
        #_color = getColor(frame, cid)
        plt.plot(xloc, yloc, marker = 'o', color = 'b', linestyle = 'none')
        areInteracting(cid, frame)
        #showTrail(cid, frame)
        #plt.quiver(xloc, yloc, xloc+0.5*math.cos(theta_val), yloc+0.5*math.sin(theta_val), angles = 'xy')
        #plt.plot([xloc, xloc+100*math.cos(theta_val)], [yloc, yloc+100*math.sin(theta_val)], 'k-')
    plt.tight_layout()
    plt.savefig(getName(frame))
    plt.close()
    return(getName(frame))

**Before running parallel code:** Run the function for a single frame to test if your code works
- It seems that we get a key error for cid=49. By printing pos, you can confirm that the dictionary is not initialized for cid 49
- Perhaps error in 3rd code block above, where you write `for i in range(N-1):`

In [20]:
createPlot(2)

'00002.png'

**Comments:** When developing code, always run serially and debug before running in parallel. If you have a bug in your code, the parallel computation will throw an error that is not very helpful.

In [21]:
results = Parallel(n_jobs=num_cores)(delayed(createPlot)(frame) for frame in tpoints)

Encode movie

In [22]:
! chmod +x make_png_movie.sh
! ./make_png_movie.sh

Creating Movie - Frames: 250 Estimated Length: 25 seconds
ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex

In [23]:
#Rename movie file

In [24]:
! mv my_movie.mp4 SimulationResults/periodic.mp4

Delete images

In [25]:
! rm -rf *.png

In [26]:
HTML("""
<video width="640" height="640" controls>
  <source src="SimulationResults/periodic.mp4" type="video/mp4">
</video>
""")